## Melita FC Expected Goals Tracker Website

In [1]:
import pandas as pd
import matplotlib as plt
import dash
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
%matplotlib inline

In [35]:
df = pd.read_csv('game_data.csv')
df['Time'] = pd.to_numeric(df['Time'])
df['Date'] = pd.to_datetime(df['Date'],dayfirst=True)
df.set_index('Time',inplace=True)
df

,Mel_Reg,Opp_Reg,Type,Opposition,H_A,Date
Time,,,,,,
1.0,N,N,NaN,SA,A,2020-12-14
2.0,N,N,NaN,SA,A,2020-12-14
3.0,D,N,Cross,SA,A,2020-12-14
4.0,N,N,NaN,SA,A,2020-12-14
5.0,N,N,NaN,SA,A,2020-12-14
...,...,...,...,...,...,...
90.0,D,N,FK,LIJ,A,2020-12-07
91.0,N,N,NaN,LIJ,A,2020-12-07
92.0,N,N,NaN,LIJ,A,2020-12-07


In [36]:
def assign_xG(df):
    d = {'N':0,'F':0.01,'E':0.05,'D':0.09,'C':0.15,'B':0.24,'A':0.4,'A+':0.6,'Corner':0.18}
    
    Melita = df.columns[0]
    Opposition = df.columns[1]
    
    df[Melita + '_xG'] = df[Melita].apply(lambda x: d.get(x)).astype('float')
    df[Opposition + '_xG'] = df[Opposition].apply(lambda x: d.get(x)).astype('float')
    
    
    df2 = df.groupby('Type').sum()
    
    #Home Team
    #sns.barplot(x=df2.index,y=df2[home + ' xG'])
    #plt.savefig(home+'.jpeg')
    
    #Away Team
    #sns.barplot(x=df2.index,y=df2[away + ' xG'])
    #plt.savefig(away+'.jpeg')
    
    
    
    #plot xG comparison
    #sns.set_style("white")
    #sns.lineplot(x=df.index,y=df[home + ' xG'],
            # data=df,color = 'r',label=home)
    #sns.lineplot(x=df.index,y=away,
             #data=df,color='b',label = away)
    #plt.savefig('xG_comp.jpeg')

    
    
    
    return df


In [37]:
assign_xG(df)

,Mel_Reg,Opp_Reg,Type,Opposition,H_A,Date,Mel_Reg_xG,Opp_Reg_xG
Time,,,,,,,,
1.0,N,N,NaN,SA,A,2020-12-14,0.00,0.0
2.0,N,N,NaN,SA,A,2020-12-14,0.00,0.0
3.0,D,N,Cross,SA,A,2020-12-14,0.09,0.0
4.0,N,N,NaN,SA,A,2020-12-14,0.00,0.0
5.0,N,N,NaN,SA,A,2020-12-14,0.00,0.0
...,...,...,...,...,...,...,...,...
90.0,D,N,FK,LIJ,A,2020-12-07,0.09,0.0
91.0,N,N,NaN,LIJ,A,2020-12-07,0.00,0.0
92.0,N,N,NaN,LIJ,A,2020-12-07,0.00,0.0


In [ ]:
app = dash.Dash(__name__)
server = app.server
team_names = df['Date'].dt.date.unique()
team_names.sort()
app.layout = html.Div([
    html.Div([dcc.Dropdown(id='select_team', options=[{'label': i, 'value': i} for i in team_names],
                           value='SA', style={'width': '140px'})]),
    dcc.Graph("line-chart", config={'displayModeBar': True})])
@app.callback(
    Output("line-chart", 'figure'),
    [Input('select_team', 'value')]
)

def update_graph(grpname):
    import plotly.express as px
    fig = px.line(df[df.Date == grpname], x=df[df.Date == grpname].index, 
                   y=[df[df.Date == grpname].Opp_Reg_xG,df[df.Date == grpname].Mel_Reg_xG],labels={'x':'Time','value':'xG'})
    fig.update_layout(title_text="Expected Goals Tracker", legend_title="Teams")
    d = {'2020-12-14':'St Andrews','2020-12-07': 'Lija','2020-12-21':'Valletta'}
    nameswap = {'wide_variable_0': d.get(grpname),'wide_variable_1': 'Melita'}
    
    for i, dat in enumerate(fig.data):
        for elem in dat:
            if elem == 'name':
                fig.data[i].name = nameswap[fig.data[i].name]
    return fig
    

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Jan/2021 18:38:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2021 18:38:07] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2021 18:38:07] "GET /_dash-dependencies HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\aidan\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\aidan\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\aidan\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\aidan\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\aidan\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\aidan\Anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\aidan\Anaconda3\lib\site-packages\dash\dash.py", line 1076, in dispatch
    response.set_d

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\aidan\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\aidan\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\aidan\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\aidan\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\aidan\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\aidan\Anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\aidan\Anaconda3\lib\site-packages\dash\dash.py", line 1076, in dispatch
    response.set_d

127.0.0.1 - - [02/Jan/2021 18:38:07] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [02/Jan/2021 18:38:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2021 18:38:16] "POST /_dash-update-component HTTP/1.1" 200 -
